In [ ]:
# --- COLAB SETUP (run this first) ---
import os, sys

REPO_URL = "https://github.com/mrc-rossoni/surface-book"
REPO_DIR = "surface-book"

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL}

sys.path.append(REPO_DIR)
!pip install -q -r {REPO_DIR}/requirements.txt
sys.path.append("surface-book/code")
print("✅ Setup complete")

# Simple Bézier Curve - De Casteljau

In [ ]:
# Import Library

import numpy as np
import matplotlib.pyplot as plt

# Define 4 control points in 2D
P = np.array([
    [0.0, 0.0],
    [0.2, 1.2],
    [0.8, 1.0],
    [1.0, 0.0]
])

# discretize the parameter space in 100 evenly spaced points
t_values = np.linspace(0, 1, 100)

# -------- De Casteljau (single-point evaluation) -------- #
def de_casteljau(control_points, t):
    """
    Evaluate a Bézier curve at parameter t using De Casteljau algorithm.
    control_points: array of shape (n+1, 2)
    t: scalar in [0,1]
    returns: point of shape (2,)
    """
    Q = np.asarray(control_points, dtype=float).copy()
    t = float(t)
    n = len(Q) - 1

    for r in range(1, n + 1):
        for i in range(n - r + 1):
            Q[i] = (1 - t) * Q[i] + t * Q[i + 1]

    return Q[0]

# -------- Evaluate curve for all t -------- #
C = np.array([de_casteljau(P, t) for t in t_values])  # shape (100, 2)

# -------- Plot -------- #
plt.figure(figsize=(7, 5))
plt.plot(P[:, 0], P[:, 1], "--o", label="Control polygon")
plt.plot(C[:, 0], C[:, 1], "-", linewidth=2, label="Bézier curve")
plt.axis("equal")
plt.grid(True)
plt.legend()
plt.show()


## Compexity Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# ------------------------------------------------------------
# De Casteljau evaluation (single point)
# ------------------------------------------------------------
def de_casteljau(control_points, t):
    Q = np.asarray(control_points, dtype=float).copy()
    t = float(t)
    n = len(Q) - 1
    for r in range(1, n + 1):
        for i in range(n - r + 1):
            Q[i] = (1 - t) * Q[i] + t * Q[i + 1]
    return Q[0]

# ------------------------------------------------------------
# Benchmark: single evaluation time vs number of control points
# ------------------------------------------------------------
def benchmark_single_eval(n_points_list, t=0.5, repeats=50):
    """
    Returns median time (ms) to evaluate De Casteljau at a single t.
    Control points are generated once per n to reduce noise.
    """
    times_ms = []

    for n in n_points_list:
        control_points = np.random.rand(n, 2)

        # Warm-up
        de_casteljau(control_points, t)

        samples = []
        for _ in range(repeats):
            start = time.perf_counter_ns()
            de_casteljau(control_points, t)
            end = time.perf_counter_ns()
            samples.append((end - start) * 1e-6)  # ns -> ms

        times_ms.append(np.median(samples))

    return np.array(times_ms)

# ------------------------------------------------------------
# Fit power law: y ≈ a * x^b  (log-log regression)
# ------------------------------------------------------------
def fit_power_law(x, y):
    logx = np.log(x)
    logy = np.log(y)
    b, loga = np.polyfit(logx, logy, 1)
    a = np.exp(loga)
    return a, b

# ------------------------------------------------------------
# Fit theory model y ≈ k * n^2
# (best k via least squares)
# ------------------------------------------------------------
def fit_theory_constant_n2(n, y):
    """
    Find k minimizing || y - k*n^2 ||^2
    """
    x = n**2
    k = np.dot(x, y) / np.dot(x, x)
    return k

# ------------------------------------------------------------
# Distance between theory and experiment
# ------------------------------------------------------------
def theory_distance_metrics(y_meas, y_theory):
    """
    Computes:
      - relative error (pointwise)
      - RMSE in linear space
      - RMSE in log space (recommended for complexity models)
    """
    rel_err = np.abs(y_meas - y_theory) / y_meas
    rmse = np.sqrt(np.mean((y_meas - y_theory) ** 2))
    rmse_log = np.sqrt(np.mean((np.log(y_meas) - np.log(y_theory)) ** 2))
    return rel_err, rmse, rmse_log

# ------------------------------------------------------------
# Plot: linear + log-log + theory distance
# ------------------------------------------------------------
def plot_casteljau_complexity(max_points=60, t=0.5, repeats=80):
    n = np.arange(2, max_points + 1)
    y = benchmark_single_eval(n, t=t, repeats=repeats)  # ms

    # Fit power law y ≈ a*n^b
    a, b = fit_power_law(n, y)
    y_fit = a * n**b

    # Fit theory y ≈ k*n^2
    k = fit_theory_constant_n2(n, y)
    y_theory = k * n**2

    # Distance metrics
    rel_err, rmse, rmse_log = theory_distance_metrics(y, y_theory)

    # --------------------------------------------------------
    # 1) Linear plot (with fitted power law and theory n^2)
    # --------------------------------------------------------
    plt.figure(figsize=(10, 6))
    plt.plot(n, y, "o", label="Measured (median)")
    plt.plot(n, y_fit, "-", label=f"Fit: {a:.2e} · n^{b:.2f}")
    plt.plot(n, y_theory, "--", label=f"Theory: {k:.2e} · n^2")
    plt.xlabel("Number of Control Points (n)")
    plt.ylabel("Time per evaluation (ms)")
    plt.title("De Casteljau — Single Evaluation Time (Linear scale)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # --------------------------------------------------------
    # 2) Log-log plot (best for complexity)
    # --------------------------------------------------------
    plt.figure(figsize=(10, 6))
    plt.loglog(n, y, "o", label="Measured (median)")
    plt.loglog(n, y_fit, "-", label=f"Fit: n^{b:.2f}")
    plt.loglog(n, y_theory, "--", label="Theory: n^2")
    plt.xlabel("Number of Control Points (n) [log]")
    plt.ylabel("Time per evaluation (ms) [log]")
    plt.title("De Casteljau — Single Evaluation Time (Log–log scale)")
    plt.grid(True, which="both")
    plt.legend()
    plt.show()

    # --------------------------------------------------------
    # 3) Relative error vs n
    # --------------------------------------------------------
    plt.figure(figsize=(10, 6))
    plt.plot(n, rel_err * 100, "o-")
    plt.xlabel("Number of Control Points (n)")
    plt.ylabel("Relative error vs k·n² (%)")
    plt.title("Distance between experiment and O(n²) theory")
    plt.grid(True)
    plt.show()

    # Print summary
    print("=== Complexity fit (power law) ===")
    print(f"Measured ~ a * n^b with:")
    print(f"  a = {a:.3e} ms")
    print(f"  b = {b:.3f}")

    print("\n=== Best matching O(n²) theory ===")
    print(f"Best k for k·n²: k = {k:.3e} ms")

    print("\n=== Distance metrics (theory vs experiment) ===")
    print(f"RMSE (linear space): {rmse:.3e} ms")
    print(f"RMSE (log space):    {rmse_log:.3e} (dimensionless)")
    print(f"Mean relative error: {np.mean(rel_err)*100:.2f} %")

# ------------------------------------------------------------
# Run
# ------------------------------------------------------------
plot_casteljau_complexity(max_points=30, t=0.5, repeats=3)


# TBD

In [ ]:
u = np.linspace(0, 1, 200)
C = bezier_eval(P, u)

plt.figure()
plt.plot(P[:,0], P[:,1], "--o", label="Control polygon")
plt.plot(C[:,0], C[:,1], label="Bezier curve")
plt.axis("equal")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
u_test = 0.35
c1 = bezier_eval(P, np.array([u_test]))[0]
c2 = de_casteljau(P, u_test)

print("Bernstein:", c1)
print("De Casteljau:", c2)
print("Difference:", np.linalg.norm(c1 - c2))


Exercises section (markdown cell)

Change control points to create inflection points

Increase degree with random control points and test stability

Plot curve + convex hull polygon

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, Play, jslink, HBox, VBox

import time

# Enable inline plotting
%matplotlib inline

# Linear interpolation (lerp) function
def lerp(p0, p1, t):
    """Perform linear interpolation between points p0 and p1 using parameter t."""
    return (1 - t) * np.array(p0) + t * np.array(p1)

# De Casteljau's algorithm using only lerp
def de_casteljau(points, t):
    """
    Computes the point on the Bézier curve at parameter t by recursively applying lerp.
    
    Parameters:
      points: List of control points (each a tuple or list of coordinates)
      t: Parameter value in [0, 1]
    
    Returns:
      point: The computed point on the Bézier curve at t.
      levels: A list of lists, where each inner list contains the intermediate points at that recursion level.
    """
    points = [np.array(p) for p in points]
    levels = [points]  # level 0: the original control points
    while len(points) > 1:
        points = [lerp(points[i], points[i + 1], t) for i in range(len(points) - 1)]
        levels.append(points)
    return points[0], levels

In [ ]:
def plot_de_casteljau(t):
    """
    Plots the control polygon, intermediate levels, and the Bézier curve traced from t=0 to t.
    """
    colors = ['red', 'green', 'blue', 'orange', 'black']
    cp = np.array(control_points)
    # Compute current point and intermediate levels for t
    point, levels = de_casteljau(control_points, t)
    
    plt.figure(figsize=(15, 10))
    
    # Plot the control polygon
    #plt.plot(cp[:, 0], cp[:, 1], 'k--', label='Control Polygon')
    plt.plot(cp[:, 0], cp[:, 1], 'ko')
    
    # Plot each intermediate level
    for i, level in enumerate(levels[:-1]):  # skip the final level (single point)
        pts = np.array(level)
        plt.plot(pts[:, 0], pts[:, 1], 'o-', color=colors[i % len(colors)], label=f'Recursion Level {i+1}')
    
    # Trace the curve from t=0 to the current t
    if t > 0:
        t_values = np.linspace(0, t, 100)
        curve_points = np.array([de_casteljau(control_points, ti)[0] for ti in t_values])
        plt.plot(curve_points[:, 0], curve_points[:, 1], 'k-', linewidth=2, label='Traced Curve')
    
    # Highlight the final computed point on the curve
    plt.plot(point[0], point[1], 'ko', markersize=10, label=f'Curve Point t={t:.2f}')
    
    plt.title("De Casteljau's Algorithm: Geometric Construction")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.show()

##### Run the following code to get an animated view

In [ ]:
# Define control points and additional variables for the animation
control_points = [(0, 0), (1, 2), (3, 3), (4, 0)]

# Create a FloatSlider widget for t
t_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='t')

# Create a Play widget to animate t (0 to 100 corresponds to t from 0 to 1)
play = widgets.Play(value=0, min=0, max=100, step=1, description="Press play", interval=100)

# Link the Play widget to the slider using an observer with a simple transform.
def update_slider(change):
    t_slider.value = change['new'] / 100

play.observe(update_slider, names='value')

# Display the Play widget and slider side by side.
display(HBox([play]))

# Create the interactive plot: as the slider moves, plot_de_casteljau is updated.
interact(plot_de_casteljau, t=t_slider);


# Complexity Analysis

In [ ]:
def measure_de_casteljau_time(n, t=0.5, trials=10):
    """
    Measures the average computation time of the de_casteljau algorithm for n control points.
    
    Parameters:
      n: Number of control points.
      t: Parameter for the algorithm.
      trials: Number of runs to average over.
    
    Returns:
      Average computation time in seconds.
    """
    total_time = 0.0
    for _ in range(trials):
        # Generate n random 2D control points.
        control_points = [np.random.rand(2) for _ in range(n)]
        start_time = time.perf_counter()
        de_casteljau(control_points, t)
        end_time = time.perf_counter()
        total_time += (end_time - start_time)
    return total_time / trials


def measure_curve_time(n, discretization=100, trials=100):
    """
    Measures the average computation time to compute a full Bézier curve 
    (evaluated at a specified number of discretization points) for a curve 
    defined by n control points.
    
    Parameters:
      n: Number of control points.
      discretization: Number of points along the curve (t values).
      trials: Number of trials for averaging.
    
    Returns:
      Average computation time (in seconds) for computing the full curve.
    """
    total_time = 0.0
    for _ in range(trials):
        control_points = [np.random.rand(2) for _ in range(n)]
        t_values = np.linspace(0, 1, discretization)
        start_time = time.perf_counter()
        for t in t_values:
            de_casteljau(control_points, t)
        end_time = time.perf_counter()
        total_time += (end_time - start_time)
    return total_time / trials

def plot_computation_time(max_points=50, t=0.5, trials=100):
    """
    Plots the average computation time of De Casteljau's algorithm vs. the number of control points,
    with computation time converted to milliseconds (dividing seconds by 1000).
    
    Parameters:
      max_points: Maximum number of control points to test (starting from 2).
      t: Parameter for the algorithm.
      trials: Number of runs per test for averaging.
    """
    num_points = list(range(2, max_points + 1))
    # Collect times as a list and convert to a numpy array for element-wise operations.
    times = np.array([measure_de_casteljau_time(n, t, trials) for n in num_points])
    
    plt.figure(figsize=(10, 6))
    plt.plot(num_points, times * 1000, marker='o')  # times in milliseconds
    plt.xlabel('Number of Control Points')
    plt.ylabel('Average Computation Time (ms)')
    plt.title("De Casteljau's Algorithm Computation Time vs. Number of Control Points")
    plt.grid(True)
    plt.show()

def plot_computation_time_points(discretization_range, cps_values, trials):
    # Dictionary to store computation times (in milliseconds)
    results = {}
    
    for cps in cps_values:
        times = []
        for d in discretization_range:
            avg_time = measure_curve_time(cps, discretization=d, trials=10)
            times.append(avg_time * 1000)  # convert seconds to milliseconds
        results[cps] = times
    
    # Plot the results: x-axis is the number of discretization points; one curve per control point count
    plt.figure(figsize=(10, 6))
    for cps in cps_values:
        plt.plot(list(discretization_range), results[cps], marker='o', label=f'{cps} control points')
    plt.xlabel('Number of Discretization Points')
    plt.ylabel('Average Computation Time (milliseconds)')
    plt.title('Computation Time vs. Discretization Points (per Bézier Curve)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
# Run the function to display the plot.
plot_computation_time(max_points=50, t=0.5, trials=100)

In [ ]:
# Define a range for the number of discretization points
discretization_range = range(10, 201, 10)  # 10, 20, ... 200

# Define the control point counts to test
cps_values = [3, 4, 5]

plot_computation_time_points(discretization_range, cps_values, trials= 100)